In [3]:
import pyupbit as up
import datetime 
import numpy as np 
import pandas as pd 
import time 
import random 
import os 
import math  
import matplotlib.pyplot as plt

In [4]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m
        
    y = np.asarray(y)

    return np.maximum(y-tmax,0.)



# Demo Run 

Check if the model runs correctly on dummy data

In [5]:
import numpy as np
import pandas as pd
# from utils import *

class OLMAR:
    def __init__(self, epsilon, window_size):
        self.epsilon = epsilon
        self.window_size = window_size
        self.cumulative_wealth = 1.0

    def olmar(self, current_portfolio, predicted_returns, window_size):
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_right = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_right/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_proj(new_portfolio.tolist())
        return new_portfolio

    def predict(self, current_portfolio, current_return, previous_prices):
        # calculate cumulative wealth
        self.cumulative_wealth *= np.sum([x*y for x,y in zip(current_portfolio, current_return)])
        m = len(previous_prices[0]) # number of assets
        predicted_returns = []
        for i in range(m): # for each asset, calculate predicted return
            temp = 0
            for j in range(self.window_size):
                temp += previous_prices[j][i]
            predicted_return = (1/previous_prices[0][i]) * (1/self.window_size) * temp
            predicted_returns.append(predicted_return)

        new_portfolio = self.olmar(current_portfolio, predicted_returns, self.window_size)
        return new_portfolio

    def get_cumulative_wealth(self):
        return self.cumulative_wealth

olmar = OLMAR(10, 5) # epsilon = 10, w = 5
'''
current time step is t
since our window size is w = 5, we get the previous timesteps t,t-1,t-2,t-3,t-4
let us assume we have 4 assets (m = 4)
we repeat the below process for n (indefinite number of) timesteps
'''
current_portfolio = [0.25, 0.25, 0.25, 0.25]
previous_prices = [[100,120,130,110],  # t
                   [110,121,90,80], # t-1
                   [120,140,85,75], # t-2
                   [140,200,75,51], # t-3
                   [300,240,80,80]] # t-4
current_returns = [100/110, 120/121, 130/90, 110/80]


new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)

print("portfolio at time t = {}".format(current_portfolio))
print("predicted portfolio at time t+1 = {}".format(new_portfolio))
print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))


portfolio at time t = [0.25, 0.25, 0.25, 0.25]
predicted portfolio at time t+1 = [1. 0. 0. 0.]
cumulative wealth = 1.180067722681359


# Get Price Information

Get prices from 2020/12/29 to 2021/06/15 for every 4 hour periods

In [84]:
def get_dataframes(coin_name, interval): 
    january = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210131")  
    february = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210228") 
    march = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210331") 
    april = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210430") 
    may = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210531") 
    june = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210616") 
    df = pd.concat([january, february, march, april, may, june]) 
    df.drop_duplicates(keep=False,inplace=True) 
    return df['close'].values 

In [90]:
btc = get_dataframes("BTC", "minute240") 

In [91]:
eth = get_dataframes("ETH", "minute240") 

In [92]:
ada = get_dataframes("ADA", "minute240") 

In [94]:
xrp = get_dataframes("XRP", "minute240") 

In [95]:
dot = get_dataframes("DOT", "minute240") 

In [100]:
price_data = [btc, eth, ada, xrp, dot] 

# Backtest without commissions

In [103]:
epsilon = 10 
m = 5
w = 5
olmar = OLMAR(epsilon, w)

cnt = 1 
current_portfolio = [1/m for i in range(m)] 
for i in range(w-1,len(btc)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
    
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price)  
        
    previous_prices = np.asarray(previous_prices)  
    
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)  
    print("Iteration = {}".format(cnt))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))  
    
    print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))
    print()
    cnt += 1 
    current_portfolio = new_portfolio 



Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 0.9923484008782484

Iteration = 2
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 0.8953187794590419

Iteration = 3
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.0937884596346916

Iteration = 4
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 1.111537784210417

Iteration = 5
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.1230124276752904

Iteration = 6
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.1280258760131265

Iteration = 7
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.11

Iteration = 172
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.6019482675925405

Iteration = 173
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.608341447679854

Iteration = 174
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.5900752188589584

Iteration = 175
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.4630065728594304

Iteration = 176
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.38401795507594

Iteration = 177
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.3943208182650908

Iteration = 178
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1

portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.871122631008929

Iteration = 347
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.8427722881148545

Iteration = 348
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.8427722881148545

Iteration = 349
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.7860716023267051

Iteration = 350
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.7674666898024687

Iteration = 351
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.7583911227174753

Iteration = 352
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.7475004422154

predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 2.778757988185898

Iteration = 536
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.842637482167183

Iteration = 537
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 2.9581918513609713

Iteration = 538
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.0152630767891573

Iteration = 539
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 2.9486799804562738

Iteration = 540
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 2.9867274640750643

Iteration = 541
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.0036016305387654

Iteration = 542
portfolio at time t

Iteration = 681
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 4.905449581587864

Iteration = 682
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 4.93378430841292

Iteration = 683
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 4.822806628348117

Iteration = 684
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 4.738983061490659

Iteration = 685
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 4.887740377322205

Iteration = 686
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 4.935602331526497

Iteration = 687
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 4.953

# With commissions

In [104]:
epsilon = 10 
m = 5
w = 5
# create OLMAR instance 
olmar = OLMAR(epsilon, w)

iterations = 1 
current_portfolio = [1/m for i in range(m)] 
for i in range(w-1,len(btc)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
    
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price)  
        
    previous_prices = np.asarray(previous_prices)  
    
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)  
    print("Iteration = {}".format(iterations))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))  
    
    # compare current and predicted portfolio weights to deduce buy and sell counts 
    buy_sell_cnt = 0 
    for i in range(len(current_portfolio)): 
        buy_sell_cnt += (current_portfolio[i] != new_portfolio[i]) 
    
    
    print("cumulative wealth = {}".format(olmar.get_cumulative_wealth() * np.power((1-0.05/100), buy_sell_cnt)))
    print()
    iterations += 1 
    current_portfolio = new_portfolio 



Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 0.9898700095069298

Iteration = 2
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 0.8953187794590419

Iteration = 3
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.092694944622172

Iteration = 4
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 1.1104265243106528

Iteration = 5
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.1218896960007223

Iteration = 6
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.1268981321435825

Iteration = 7
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.11

Iteration = 212
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.724219910166044

Iteration = 213
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.722143782200479

Iteration = 214
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.7621983222842386

Iteration = 215
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.7606152225360931

Iteration = 216
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.8065443152979044

Iteration = 217
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 1.8664297622138568

Iteration = 218
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 

portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.963446747397384

Iteration = 417
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 1.9510759332608574

Iteration = 418
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 2.012140141859098

Iteration = 419
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.0166560661485646

Iteration = 420
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.0377543324616165

Iteration = 421
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 2.0890879475414246

Iteration = 422
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 2.10339581038040

Iteration = 607
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.4314730109118137

Iteration = 608
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.4482311024580197

Iteration = 609
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.506770984682399

Iteration = 610
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.516210543113245

Iteration = 611
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.4739269354455162

Iteration = 612
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.5166953739074476

Iteration = 613
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 

portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 2.3479464403245047

Iteration = 814
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 2.3223164328410406

Iteration = 815
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 2.4387896061703933

Iteration = 816
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.4933699774630167

Iteration = 817
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.4835094503059447

Iteration = 818
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 2.4835094503059447

Iteration = 819
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 2.468683146002

# Just holding and doing nothing 

What if we just use a constant portfolio 

In [106]:
epsilon = 10 
m = 5
w = 5

iterations = 1 
current_portfolio = [1/m for i in range(m)]  
cumulative_wealth = 1.0 

for i in range(w-1,len(btc)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
        
    print("Iteration = {}".format(iterations))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(current_portfolio))  
    
    cumulative_wealth *= np.sum([x*y for x,y in zip(current_portfolio, current_returns)]) 

    print("cumulative wealth = {}".format(cumulative_wealth))
    print()
    iterations += 1  



Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 0.9923484008782484

Iteration = 2
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 0.9719425969254738

Iteration = 3
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 1.0369770337965147

Iteration = 4
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 1.0510334765005052

Iteration = 5
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 1.033394375522418

Iteration = 6
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 1.0208951113887232

Iteration = 7
portfolio at ti

portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.7688958844671885

Iteration = 391
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.7950500645916603

Iteration = 392
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.743738139558457

Iteration = 393
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.709964305009753

Iteration = 394
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.71686571343885

Iteration = 395
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 3.670076287056716

Iteration = 396
portfolio at time t =


Iteration = 769
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.595604306870184

Iteration = 770
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.618673578984342

Iteration = 771
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.58158083812123

Iteration = 772
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.620170885908216

Iteration = 773
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.585890594566275

Iteration = 774
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0.2, 0.2, 0.2, 0.2, 0.2]
cumulative wealth = 4.622653008880772

Iteration = 775
portfo